# Aprendizaje Profundo: Práctico 

En este práctico trabajaremos en el problema de clasificación de texto del MeLi Challenge 2019.

<img src="images/img1_practico.png"
     alt="MeLi Challenge Spanish"
     style="float: center; margin-right: 10px;"
     width=80%/>

El datasets tiene información acerca de títulos de publicaciones, categoría de los mismos, información de idioma y confiabilidad de la anotación.
Cuenta con anotaciones de títulos para 632 categorías distintas.

<img src="images/img2_practico.png"
     alt="Categorias"
     style="float: center; margin-right: 10px;"
     width=50%/>

El dataset también cuenta con una partición de test que está compuesta de 63680 de ejemplos con las mismas categorías
(aunque no necesariamente la misma distribución).

También hay datos en idioma portugues, aunque para el práctico de esta materia basta con usar uno solo de los idiomas.

<img src="images/img3_practico.png"
     alt="MeLi Challenge Portuguese"
     style="float: center; margin-right: 10px;"
     width=80%/>

## Ejercicio:
Implementar una red neuronal que asigne una categoría dado un título.
Para este práctico se puede usar cualquier tipo de red neuronal. Les que hagan solo la primera mitad de la materia,
implementarán un MLP. Quienes cursan la materia completa, deberían implementar algo más complejo, usando CNNs,
RNNs o Transformers.

<img src="images/img4_practico.png"
     alt="NN Generic Architecture"
     style="float: center; margin-right: 10px;"
     width=40%/>


Algunas consideraciones a tener en cuenta para estructurar el trabajo:

  1. Hacer un preprocesamiento de los datos (¿Cómo vamos a representar los datos de entrada y las categorías?).
  2. Tener un manejador del dataset (alguna clase o función que nos divida los datos en batches).
  3. Crear una clase para el modelo que se pueda instanciar con diferentes hiperparámetros
  4. Hacer logs de entrenamiento (reportar tiempo transcurrido, iteraciones/s, loss, accuracy, etc.). Usar MLFlow.
  5. Hacer un gráfico de la función de loss a lo largo de las epochs. MLFlow también puede generar la gráfica.
  6. Reportar performance en el conjunto de test con el mejor modelo entrenado. La métrica para reportar será balanced accuracy ([Macro-recall](https://peltarion.com/knowledge-center/documentation/evaluation-view/classification-loss-metrics/macro-recall)).

## Ejercicios opcionales:
  1. Se puede tomar una subconjunto del train para armar un set de validation para probar configuraciones, arquitecturas o hiperparámetros (cuidado con la distribución de este conjunto). **No usar el conjunto de test para ajustar hiperparámetros.**
  2. Se puede usar todos los datos (spanish & portuguese) para hacer un modelo multilingual para la tarea. ¿Qué cosas tendrían que tener en cuenta para ello?

Adicionalmente, se pide un reporte de los experimentos y los procesos que se llevaron a cabo (en el README.md de su repositorio correspondiente). 
No se evaluará la performance de los modelos, sino el proceso de tomar el problema e implementar una solución con aprendizaje profundo.


In [1]:
import gzip
import json
import nltk
import pandas as pd
from torch.utils.data import Dataset, DataLoader, IterableDataset

from gensim import corpora
from gensim.parsing import preprocessing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors


from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
tqdm.pandas()

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df_train = pd.read_json(f"./data/meli-challenge-2019/spanish.train.jsonl.gz", lines=True)
   


df_train.to_csv (r'./data/meli-challenge-2019/meli_train.csv', index = None)

In [8]:
df_validation = pd.read_json(f"./data/meli-challenge-2019/spanish.validation.jsonl.gz", lines=True)

df_validation.to_csv (r'./data/meli-challenge-2019/meli_validation.csv', index = None)

In [10]:
df_test =  pd.read_json(f"./data/meli-challenge-2019/spanish.test.jsonl.gz", lines=True)  

df_test.to_csv (r'./data/meli-challenge-2019/meli_test.csv', index = None)

In [20]:
df_token = pd.read_json(f"./data/meli-challenge-2019/spanish_token_to_index.json.gz", lines=True)

In [22]:
df_token

,barbies,casita,muñecas,pintadas,cromado,holográfico,neceser,asiento,chevrolet,funda,...,chorros,budadá,cáp,sebium,hlertfx,promix,acrilab,galapagos,[PAD],[UNK]
0,50000,50001,2,3,4,5,6,7,8,9,...,49992,49993,49994,49995,49996,49997,49998,49999,0,1


In [15]:
df_test.head()

,language,label_quality,title,category,split,tokenized_title,data,target,n_labels,size
0,spanish,reliable,Mochilas Maternales Bolsos Bebe Simil Cuero Ma...,DIAPER_BAGS,test,"[mochilas, maternales, bolsos, bebe, simil, cu...","[5650, 5271, 5268, 915, 2724, 375, 37363]",318,632,63680
1,spanish,reliable,Bolso Maternal/bebe Incluye Cambiador + Correa...,DIAPER_BAGS,test,"[bolso, maternal, bebe, incluye, cambiador, co...","[502, 2742, 915, 3031, 2740, 1840, 4635]",318,632,63680
2,spanish,reliable,Mochila Maternal Land + Gancho Envio Gratis-cc,DIAPER_BAGS,test,"[mochila, maternal, land, gancho, envio, gratis]","[337, 2742, 2741, 3303, 211, 1429]",318,632,63680
3,spanish,reliable,Bolso Maternal Moderno Con Cambiador Y Correa ...,DIAPER_BAGS,test,"[bolso, maternal, moderno, cambiador, correa, ...","[502, 2742, 2983, 2740, 1840, 476, 2990]",318,632,63680
4,spanish,reliable,Bolso Maternal Moderno Con Cambiador Y Correa ...,DIAPER_BAGS,test,"[bolso, maternal, moderno, cambiador, correa, ...","[502, 2742, 2983, 2740, 1840, 4635]",318,632,63680


Las bases de datos ya estan tokenizadas. Vemos que en la columna data nos aparece el nro correspondiente a sus respectivos titulos tokenizados. la base de datos token to index, nos remite directamente nro por nombre. 

In [16]:
df_trainR = df_train.drop(columns=['language', 'label_quality', 'title',
                               'category', 'split', 'tokenized_title',
                               'n_labels', 'size'])

In [17]:
df_ValiR = df_validation.drop(columns=['language', 'label_quality', 'title',
                               'category', 'split', 'tokenized_title',
                               'n_labels', 'size'])

In [18]:
df_testR = df_test.drop(columns=['language', 'label_quality', 'title',
                               'category', 'split', 'tokenized_title',
                               'n_labels', 'size'])

In [19]:
df_trainR.head()

,data,target
0,"[50001, 2, 50000, 3]",0
1,"[6, 4, 5]",1
2,"[9, 7, 10, 8]",2
3,"[11, 13, 12, 14]",3
4,"[15, 19, 17, 18, 16, 1, 1, 1]",4


In [16]:
y = df_train.target
X = df_train.drop('target',axis=1)

In [17]:
y

0            0
1            1
2            2
3            3
4            4
          ... 
4895275     28
4895276     24
4895277     74
4895278     74
4895279    388
Name: target, Length: 4895280, dtype: int64

In [23]:
data = df_trainR.data

lista_num_palabras = []
for id_palabras in data:
    lista_num_palabras += id_palabras
    

print('Cantidad de etiquetas:', len(lista_num_palabras))
print('Cantidad de etiquetas diferentes:', len(set(lista_num_palabras)))
print('Valor máximo:', max(lista_num_palabras))
print('Valor mínimo:', min(lista_num_palabras))

Cantidad de etiquetas: 26129139
Cantidad de etiquetas diferentes: 50001
Valor máximo: 50001
Valor mínimo: 1


In [25]:
import numpy as np
labels = df_trainR.target.unique()
print(f'{labels.shape[0]} etiquetas ordenadas como se muestra a continuación:')
np.sort(labels)

632 etiquetas ordenadas como se muestra a continuación:


array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

DATASET

In [46]:
dataset=df_train

In [62]:
train_dataset = df_train

In [63]:
test_dataset = df_validation

In [88]:
class PadSequences:
    def __init__(self, pad_value=0, max_length=None, min_length=1):
        assert max_length is None or min_length <= max_length
        self.pad_value = pad_value
        self.max_length = max_length
        self.min_length = min_length

    def __call__(self, items):
        data, target = list(zip(*[(item["data"], item["target"]) for item in items]))
        seq_lengths = [len(d) for d in data]

        if self.max_length:
            max_length = self.max_length
            seq_lengths = [min(self.max_length, l) for l in seq_lengths]
        else:
            max_length = max(self.min_length, max(seq_lengths))

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
            
        return {
            "data": torch.LongTensor(data),
            "target": torch.FloatTensor(target)
        }

In [108]:
class RawDataProcessor:
    def __init__(self, 
                 dataset, 
                 ignore_header=True, 
                 filters=None, 
                 vocab_size=50000):
        if filters:
            self.filters = filters
        else:
            self.filters = [
                lambda s: s.lower(),
                preprocessing.strip_tags,
                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
                preprocessing.strip_numeric,
                preprocessing.remove_stopwords,
                preprocessing.strip_short,
            ]
        
        # Create dictionary based on all the reviews (with corresponding preprocessing)
        # https://radimrehurek.com/gensim/corpora/dictionary.html
        self.dictionary = corpora.Dictionary(
            dataset["data"].map(self._preprocess_string).tolist()
        )
        # Filter the dictionary with extremos words
        # https://tedboy.github.io/nlps/generated/generated/gensim.corpora.Dictionary.filter_extremes.html?highlight=filter_extrem
        self.dictionary.filter_extremes(no_below=2, no_above=1, keep_n=vocab_size)
        
        # Make the indices continuous after some words have been removed
        # https://tedboy.github.io/nlps/generated/generated/gensim.corpora.Dictionary.compactify.html
        self.dictionary.compactify()
        
        # Add a couple of special tokens
        self.dictionary.patch_with_special_tokens({
            "[PAD]": 0,
            "[UNK]": 1
        })
        self.idx_to_target = sorted(dataset["category"].unique())
        self.target_to_idx = {t: i for i, t in enumerate(self.idx_to_target)}


    def _preprocess_string(self, string):
        # https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.preprocess_string:~:text=gensim.parsing.preprocessing.preprocess_string
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _sentence_to_indices(self, sentence):
      # https://radimrehurek.com/gensim/corpora/dictionary.html#:~:text=doc2idx(document,via%20unknown_word_index.
        return self.dictionary.doc2idx(sentence, unknown_word_index=1)
    
    def encode_data(self, data):
        return self._sentence_to_indices(self._preprocess_string(data))
    
    def encode_target(self, target):
        return self.target_to_idx[target]
    
    def __call__(self, item):
        if isinstance(item["data"], str):
            data = self.encode_data(item["data"])
        else:
            data = [self.encode_data(d) for d in item["data"]]
        
        if isinstance(item["target"], str):
            target = self.encode_target(item["target"])
        else:
            target = [self.encode_target(t) for t in item["target"]]
        
        return {
            "data": data,
            "target": target
        }

In [109]:
class MyDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform
    
    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, item):
        if torch.is_tensor(item):
            item = item.to_list()
        
        item = {
            "data": self.dataset.loc[item, "title"],
            "target": self.dataset.loc[item, "category"]
        }
        
        if self.transform:
            item = self.transform(item)
        
        return item

In [110]:
preprocess = RawDataProcessor(df_train)
train_dataset = MyDataset(df_train, transform=preprocess)
preprocess = RawDataProcessor(df_validation)

test_dataset = MyDataset(df_validation, transform=preprocess)

print(f"Datasets loaded with {len(train_dataset)} training elements and {len(test_dataset)} test elements")
print(f"Sample train element:\n{train_dataset[0]}")

TypeError: decoding to str: need a bytes-like object, list found

In [92]:
class PadSequences:
    def __init__(self, pad_value=0, max_length=None, min_length=1):
        assert max_length is None or min_length <= max_length
        self.pad_value = pad_value
        self.max_length = max_length
        self.min_length = min_length

    def __call__(self, items):
        data, target = list(zip(*[(item["data"], item["target"]) for item in items]))
        seq_lengths = [len(d) for d in data]

        if self.max_length:
            max_length = self.max_length
            seq_lengths = [min(self.max_length, l) for l in seq_lengths]
        else:
            max_length = max(self.min_length, max(seq_lengths))

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
            
        return {
            "data": torch.LongTensor(data),
            "target": torch.FloatTensor(target)
        }

In [93]:
pad_sequences = PadSequences()
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True,
                          collate_fn=pad_sequences, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False,
                         collate_fn=pad_sequences, drop_last=False)

In [94]:
class MyClassifier(nn.Module):
    def __init__(self, 
                 pretrained_embeddings_path, 
                 dictionary,
                 vector_size,
                 freeze_embedings):
        super().__init__()
        embeddings_matrix = torch.randn(len(dictionary), vector_size)
        embeddings_matrix[0] = torch.zeros(vector_size)
        with gzip.open(pretrained_embeddings_path, "rt") as fh:
            for line in fh:
                word, vector = line.strip().split(None, 1)
                if word in dictionary.token2id:
                    embeddings_matrix[dictionary.token2id[word]] =\
                        torch.FloatTensor([float(n) for n in vector.split()])
        self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                                       freeze=freeze_embedings,
                                                       padding_idx=0)
        self.hidden1 = nn.Linear(vector_size, 128)
        self.hidden2 = nn.Linear(128, 128)
        self.output = nn.Linear(650, 1)
        self.vector_size = vector_size
    
    def forward(self, x):
        x = self.embeddings(x)
        x = torch.mean(x, dim=1)
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = torch.sigmoid(self.output(x))
        return x

In [95]:
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_layer1 = nn.Linear(32 * 32 * 3, 512)
        self.hidden_layer2 = nn.Linear(512, 256)
        self.output_layer = nn.Linear(256, 10)
    
    def forward(self, x: torch.Tensor):
        x = self.hidden_layer1(x)  # Go through hidden layer 1
        x = F.relu(x)  # Activation Function layer 1
        x = self.hidden_layer2(x) # Go through hidden layer 2
        x = F.relu(x)  # Activation Function layer 2
        x = self.output_layer(x)  # Output Layer
        return x

model = MLP()

In [96]:
print(model)

MLP(
  (hidden_layer1): Linear(in_features=3072, out_features=512, bias=True)
  (hidden_layer2): Linear(in_features=512, out_features=256, bias=True)
  (output_layer): Linear(in_features=256, out_features=10, bias=True)
)


In [97]:
model = MLP()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [98]:
train_dataset

In [99]:
EPOCHS = 2
model.train()  # Tell the model to set itself to "train" mode.
for epoch in range(EPOCHS):  # loop over the dataset multiple times
    running_loss = 0.0
    pbar = tqdm(train_dataset)
    for i, data in enumerate(pbar, 1):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.view(inputs.shape[0], -1))
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i > 0 and i % 50 == 0:    # print every 50 mini-batches
            pbar.set_description(f"[{epoch + 1}, {i}] loss: {running_loss / 50:.4g}")
            running_loss = 0.0

  0%|          | 0/4895280 [00:00<?, ?it/s]

AttributeError: 'str' object has no attribute 'view'

# CNN


In [105]:
class IMDBReviewsDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform
    
    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, item):
        if torch.is_tensor(item):
            item = item.to_list()
        
        item = {
            "data": self.dataset.loc[item, "data"],
            "target": self.dataset.loc[item, "target"]
        }
        
        if self.transform:
            item = self.transform(item)
        
        return item

In [106]:
class RawDataProcessor:
    def __init__(self, 
                 dataset, 
                 ignore_header=True, 
                 filters=None, 
                 vocab_size=50000):
        if filters:
            self.filters = filters
        else:
            self.filters = [
                lambda s: s.lower(),
                preprocessing.strip_tags,
                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
                preprocessing.strip_numeric,
                preprocessing.remove_stopwords,
                preprocessing.strip_short,
            ]
        
        # Create dictionary based on all the reviews (with corresponding preprocessing)
        self.dictionary = corpora.Dictionary(
            dataset["data"].map(self._preprocess_string).tolist()
        )
        # Filter the dictionary and compactify it (make the indices continous)
        self.dictionary.filter_extremes(no_below=2, no_above=1, keep_n=vocab_size)
        self.dictionary.compactify()
        # Add a couple of special tokens
        self.dictionary.patch_with_special_tokens({
            "[PAD]": 0,
            "[UNK]": 1
        })
        self.idx_to_target = sorted(dataset["target"].unique())
        self.target_to_idx = {t: i for i, t in enumerate(self.idx_to_target)}

    def _preprocess_string(self, string):
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _sentence_to_indices(self, sentence):
        return self.dictionary.doc2idx(sentence, unknown_word_index=1)
    
    def encode_data(self, data):
        return self._sentence_to_indices(self._preprocess_string(data))
    
    def encode_target(self, target):
        return self.target_to_idx[target]
    
    def __call__(self, item):
        if isinstance(item["data"], str):
            data = self.encode_data(item["data"])
        else:
            data = [self.encode_data(d) for d in item["data"]]
        
        if isinstance(item["target"], str):
            target = self.encode_target(item["target"])
        else:
            target = [self.encode_target(t) for t in item["target"]]
        
        return {
            "data": data,
            "target": target
        }

In [107]:
dataset = pd.read_csv("./data/meli-challenge-2019/meli_train.csv")
preprocess = RawDataProcessor(dataset)
train_indices, test_indices = train_test_split(dataset.index, test_size=0.2, random_state=42)
train_dataset = IMDBReviewsDataset(dataset.loc[train_indices].reset_index(drop=True), transform=preprocess)
test_dataset = IMDBReviewsDataset(dataset.loc[test_indices].reset_index(drop=True), transform=preprocess)

KeyError: 0